In [1]:
import numpy as np
import pandas as pd
import jieba
import gensim
import sklearn
from gensim.models import Doc2Vec
from sklearn import *

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis



In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#del ' ' and 'URL' from list 
def clean(com):
    empt = []
    for i in com:
        if(i!=' 'and i!='URL'):
            empt.append(i)
            
    return empt


In [5]:
def trans(string):
    if (string== 'true'):
        return 1
    return 0
        

In [6]:
#change to doc2vec
def toVec(t):
    t = list(map(jieba.cut, t))
    t= list(map (list,t ))
    t= list(map (clean,t))
    t= list(map (vecModel.infer_vector,t))
    return t

In [7]:
#read the dic of one-hot
import json
with open("../dataset/one_hot.json",'r') as fp :
    inquire_one_hot = json.load(fp)
code_size  = len(inquire_one_hot)
def oneHot(string):
    string = list(map(jieba.cut, string))
    string= list(map (list,string ))
    string= list(map (clean,string))
    string = list(map(encode,string))
    return string

def encode(string):
    code = np.zeros(code_size)
    for i in string:
        code[inquire_one_hot.get(i,0)]+=1
    return code
    

In [8]:
vecModel = Doc2Vec.load("../dataset/doc2vecModel")

In [9]:
data = pd.read_csv("../dataset/merge.csv")

In [10]:
data = data.to_numpy()

In [11]:
np.random.shuffle(data)

In [12]:
x_train_doc2vec  =toVec(data[:,1])
y_train  = data[:,2]


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.842 seconds.
Prefix dict has been built succesfully.


In [13]:
x_train_one_hot = oneHot(data[:,1])
y_train  = data[:,2]


In [14]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),  
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
scoring = [ 'precision_macro', 'recall_macro', 'accuracy','f1_macro']

In [15]:
x_train = x_train_one_hot
for i,j in zip(names,classifiers):
    print(i)
    score  = sklearn.model_selection.cross_validate(j,x_train,y_train,cv=10,scoring=scoring)
    print("Accuracy: %g\tPrecision: %g\tRecall: %g\tF-score: %g" % 
      (np.mean(score['test_accuracy']),np.mean(score['test_precision_macro']),
       np.mean(score['test_recall_macro']),np.mean(score['test_f1_macro'])))
    
    
    print()
   

Nearest Neighbors
Accuracy: 0.520874	Precision: 0.675574	Recall: 0.520768	F-score: 0.514529

Linear SVM
Accuracy: 0.69277	Precision: 0.730995	Recall: 0.692888	F-score: 0.695466

RBF SVM
Accuracy: 0.328202	Precision: 0.818044	Recall: 0.327347	F-score: 0.244193

Decision Tree
Accuracy: 0.749038	Precision: 0.759561	Recall: 0.748951	F-score: 0.7508

Random Forest
Accuracy: 0.750487	Precision: 0.780504	Recall: 0.750445	F-score: 0.752521

Neural Net
Accuracy: 0.824166	Precision: 0.827155	Recall: 0.824289	F-score: 0.823771

AdaBoost
Accuracy: 0.547677	Precision: 0.666213	Recall: 0.547119	F-score: 0.537637

Naive Bayes
Accuracy: 0.754259	Precision: 0.767029	Recall: 0.754587	F-score: 0.747415

QDA
Accuracy: 0.334347	Precision: 0.643034	Recall: 0.333624	F-score: 0.275309



In [16]:
x_train = x_train_doc2vec
for i,j in zip(names,classifiers):
    print(i)
    score  = sklearn.model_selection.cross_validate(j,x_train,y_train,cv=10,scoring=scoring)
    print("Accuracy: %g\tPrecision: %g\tRecall: %g\tF-score: %g" % 
      (np.mean(score['test_accuracy']),np.mean(score['test_precision_macro']),
       np.mean(score['test_recall_macro']),np.mean(score['test_f1_macro'])))
    
    
    print()
   

Nearest Neighbors
Accuracy: 0.310157	Precision: 0.30703	Recall: 0.310242	F-score: 0.302411

Linear SVM
Accuracy: 0.317441	Precision: 0.157793	Recall: 0.316163	F-score: 0.208241

RBF SVM
Accuracy: 0.322126	Precision: 0.257941	Recall: 0.32228	F-score: 0.274237

Decision Tree
Accuracy: 0.284566	Precision: 0.284068	Recall: 0.284673	F-score: 0.283213

Random Forest
Accuracy: 0.32209	Precision: 0.322409	Recall: 0.322031	F-score: 0.317953

Neural Net
Accuracy: 0.338271	Precision: 0.307521	Recall: 0.338193	F-score: 0.305173

AdaBoost
Accuracy: 0.31873	Precision: 0.316953	Recall: 0.318546	F-score: 0.314873

Naive Bayes
Accuracy: 0.331608	Precision: 0.301339	Recall: 0.331134	F-score: 0.286414

QDA
Accuracy: 0.326247	Precision: 0.523627	Recall: 0.325427	F-score: 0.281113



In [17]:
score  = sklearn.model_selection.cross_validate(MLPClassifier(),x_train,y_train,cv=3,scoring=scoring)
print("Accuracy: %g\tPrecision: %g\tRecall: %g\tF-score: %g" % 
      (np.mean(score['test_accuracy']),np.mean(score['test_precision_macro']),
       np.mean(score['test_recall_macro']),np.mean(score['test_f1_macro'])))

Accuracy: 0.327498	Precision: 0.310165	Recall: 0.327218	F-score: 0.290516


In [18]:
import warnings
warnings.filterwarnings('ignore')